# Create Dataset

## Load Data 

In [ ]:
# General imports
import os
import sys
import re
import random
import json

# Data Science
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# NLP
import nltk
nltk.download('wordnet')
from nltk.tokenize import TweetTokenizer
from nltk.stem import SnowballStemmer
from nltk.stem.porter import PorterStemmer
from nltk import word_tokenize
from nltk.corpus import stopwords

# Transformers
from transformers import AutoTokenizer, AutoModel, TFAutoModel
import torch, re

# Display
from tqdm import tqdm
from IPython.display import clear_output
clear_output()

In [ ]:
# PARAMS
lang = 'en'
emb = 'bert'
tag_type = 'keywords'

In [ ]:
# Import utils class
sys.path.insert(0,'../')
from utils import Utils

# Instanciate utils class
utils = Utils(r'D:\Cesard\Documents\NLP', num_workers=10)

In [ ]:
# Load manual tags
with open(f'tags/manual/reddit_{lang}.json', 'r+') as file_str:
    reddit_manual_tags = json.load(file_str)
with open(f'tags/manual/tweets_{lang}.json', 'r+') as file_str:
    twitter_manual_tags = json.load(file_str)

# Load keyword tags
with open(f'tags/{tag_type}/reddit_{lang}_words.json', 'r+') as file_str:
    reddit_enhanced_tags = json.load(file_str)
with open(f'tags/{tag_type}/tweets_{lang}_words.json', 'r+') as file_str:
    twitter_enhanced_tags = json.load(file_str)

# Load Tagged data
print('Starting to load manual tagged data...')
manual_reddit_data, manual_reddit_file_names = utils.tagged_data_loader(list(reddit_manual_tags.keys()), 'reddit', lang)
manual_twitter_data, manual_twitter_file_names = utils.tagged_data_loader(list(twitter_manual_tags.keys()), 'tweets', lang)
print(f'Loaded {len(manual_twitter_data)} tagged Tweets {len(manual_reddit_data)} and tagged Reddit docs')
print('')

# Load Enhanced Tagged data
print('Starting to load keyword tagged data...')
enhanced_reddit_data, enhanced_reddit_file_names = utils.tagged_data_loader(list(reddit_enhanced_tags.keys()), 'reddit', lang)
enhanced_twitter_data, enhanced_twitter_file_names = utils.tagged_data_loader(list(twitter_enhanced_tags.keys()), 'tweets', lang)
print(f'Loaded {len(enhanced_twitter_data)} tagged Tweets {len(enhanced_reddit_data)} and tagged Reddit docs')

# Load Not Tagged data
print('Starting to load not tagged data...')
reddit_data, _, _ = utils.data_loader(lang, 'reddit', total_data=20000, max_size = None)
twitter_data, _, _ = utils.data_loader(lang, 'tweets', total_data=20000, max_size = None)
print(f'Loaded {len(reddit_data)} tagged Tweets {len(twitter_data)} and tagged Reddit docs')
print('')

In [ ]:
# Add lists
manual_tags = {**twitter_manual_tags , **reddit_manual_tags}
manual_tagged_data = manual_twitter_data + manual_reddit_data 
manual_tagged_file_names = manual_twitter_file_names + manual_reddit_file_names 

enhanced_tags = {**twitter_enhanced_tags , **reddit_enhanced_tags}
enhanced_tagged_data = enhanced_twitter_data + enhanced_reddit_data
enhanced_tagged_file_names = enhanced_twitter_file_names + enhanced_reddit_file_names

extended_data = set(twitter_data + reddit_data) - set(manual_tagged_data + enhanced_tagged_data)

## Preprocessing

In [ ]:
def preprocess_social(data, language='en'):
    
    # Creates the language dictionary
    lang_dict = {
        "en": "english",
        "es": "spanish",
        "fr": "french"
    }
    
    data = re.sub(r'http\S+', '', data)
    
    # Sets text into lowercase
    data = data.lower()
    
    # Tokenizes by word
    tk = nltk.tokenize.TweetTokenizer(preserve_case=False, reduce_len=True, strip_handles=True)
    data = tk.tokenize(data)
    
    data_temp = []
    for word in data:
        if word not in string.punctuation:
            data_temp.append(word)
    data = data_temp
    
    # Removes stopwords
    data = [token for token in data if token not in stopwords.words(lang_dict[language])]
    
    # Creates the stemmer
    stemmer = SnowballStemmer(lang_dict[language])
    
    # Stems data
    data = [stemmer.stem(token) for token in data]
    
    # Returns preprocessed text
    return data

## Create Embeddings

In [ ]:
# Load Models
model_name = "microsoft/xtremedistil-l6-h384-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name, add_prefix_space=True)
model = AutoModel.from_pretrained(model_name, output_hidden_states=False)
device = torch.device("cuda")
model.to(device)


In [ ]:
def get_bert_embedding(data):
    
    # Preprocess data
    corpus = []
    for d in data:
        corpus.append(preprocess_social, language=lang)
        
    # Array to save embeddings
    embeddings = []
    failed_doc_counter = 0
    for i, doc in enumerate(corpus):
        try:
            # Run Bert for each document
            inputs = tokenizer(doc, return_tensors="pt", is_split_into_words=True)
            inputs.to(device)
            outputs = model(**inputs)
            # CLS Token Output
            embedding = outputs['pooler_output'].detach().cpu().numpy()[0]
            # Append representation
            embeddings.append(embedding)
        except:
            failed_doc_counter += 1
    
    print(f"Created embeddings for {len(embeddings)} docs and fail to create {failed_doc_counter} embeddings")
            
    return embeddings

In [ ]:
def get_doc2vec_embeddings(data):
    print('Coming soon...')

In [ ]:
# Create manual tags matrix for testing
y_test = np.zeros((len(manual_tags), 5))

for i, file_name in enumerate(manual_tagged_file_names):
    for j, tag in enumerate(list(manual_tags[file_name].values())):
        if tag:
            y_test[i][j] = 1
            
# Create enhanced tags matrix for training
y_aux = np.zeros((len(enhanced_tagged_data) + len(extended_data), 5))

for i, file_name in enumerate(enhanced_tagged_file_names):
    for j, tag in enumerate(list(enhanced_tags[file_name].values())):
        if tag:
            y_aux[i][j] = 1

# Add tag to last position on array if not tagged
for i, y in enumerate(y_test):
    if not sum(y):
        y[i][-1] = 1

for i, y in enumerate(y_aux):
    if not sum(y):
        y[i][-1] = 1

# Get Embeddings 
if emb == 'bert':
    # Test input embeddings
    X_test = get_bert_embedding(manual_tagged_data)
    
    # Train input embeddings
    enhanced_embeddings = get_bert_embedding(enhanced_tagged_data)
    extended_embeddings = get_bert_embedding(extended_data)
    X_aux = enhanced_embeddings + extended_embeddings
    
elif emb == 'doc2vec':
    # Test input embeddings
    X_test = get_doc2vec_embedding(manual_tagged_data)
    
    # Train input embeddings
    enhanced_embeddings = get_doc2vec_embedding(enhanced_tagged_data)
    extended_embeddings = get_doc2vec_embedding(extended_data)
    X_aux = enhanced_embeddings + extended_embeddings

In [ ]:
# Split data into training and validation subsets
X_train, X_val, y_train, y_val = train_test_split(X_aux, y_aux, test_size=0.3, random_state=42)

In [ ]:
# Save data

# Input
np.save(f'X_train_{emb}_{lang}.npy', X_train)
np.save(f'X_val_{emb}_{lang}.npy', X_val)
np.save(f'X_test_{emb}_{lang}.npy', X_test)
# Tags
np.save(f'y_train_{emb}_{lang}.npy', y_train)
np.save(f'y_val_{emb}_{lang}.npy', y_val)
np.save(f'y_test_{emb}_{lang}.npy', y_test)